In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import keras
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LeakyReLU
from keras.preprocessing.image import ImageDataGenerator


In [6]:
# Load data
train_data = pd.read_csv("E:/CodeClause/Artificial_Int/Blindness_Detection/train.csv")
test_data = pd.read_csv("E:/CodeClause/Artificial_Int/Blindness_Detection/test.csv")
train_images_path = "E:/CodeClause/Artificial_Int/Blindness_Detection/train_images/"
test_images_path = "E:/CodeClause/Artificial_Int/Blindness_Detection/test_images/"

In [7]:
# Data preprocessing
def preprocess_image(image):
    image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0, 0), 30), -4, 128)
    image = cv2.resize(image, (120, 120))
    return image

def load_and_preprocess_images(image_ids, image_path):
    images = []
    for image_id in image_ids:
        image = cv2.imread(os.path.join(image_path, image_id + ".png"))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = preprocess_image(image)
        images.append(image)
    return np.array(images)



In [8]:
# Split data
trainX, testX, trainY, testY = train_test_split(train_data['id_code'], train_data['diagnosis'], test_size=0.2)

train_imgs = load_and_preprocess_images(trainX, train_images_path)
test_imgs = load_and_preprocess_images(testX, train_images_path)


In [11]:

# Model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(120, 120, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))  # 5 classes for diagnosis

model.compile(optimizer=Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [12]:
# Data augmentation
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(train_imgs)



In [14]:
# Train the model
history = model.fit_generator(datagen.flow(train_imgs, trainY, batch_size=32),
                              steps_per_epoch=len(train_imgs) / 32,
                              validation_data=(test_imgs, testY),
                              epochs=15)



Epoch 1/15


C:\Users\ABHIJEET\AppData\Local\Temp\ipykernel_9972\2320528045.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen.flow(train_imgs, trainY, batch_size=32),


91/91 [==============================] - 36s 392ms/step - loss: 1.0754 - accuracy: 0.6200 - val_loss: 61.5318 - val_accuracy: 0.5989
Epoch 2/15
91/91 [==============================] - 37s 400ms/step - loss: 0.9782 - accuracy: 0.6559 - val_loss: 110.7935 - val_accuracy: 0.4925
Epoch 3/15
91/91 [==============================] - 36s 391ms/step - loss: 0.9565 - accuracy: 0.6729 - val_loss: 82.2010 - val_accuracy: 0.6207
Epoch 4/15
91/91 [==============================] - 39s 421ms/step - loss: 0.9214 - accuracy: 0.6763 - val_loss: 91.3217 - val_accuracy: 0.6016
Epoch 5/15
91/91 [==============================] - 40s 431ms/step - loss: 0.8874 - accuracy: 0.6869 - val_loss: 85.2790 - val_accuracy: 0.6876
Epoch 6/15
91/91 [==============================] - 40s 440ms/step - loss: 0.8537 - accuracy: 0.7033 - val_loss: 97.6362 - val_accuracy: 0.5703
Epoch 7/15
91/91 [==============================] - 38s 414ms/step - loss: 0.8177 - accuracy: 0.7026 - val_loss: 43.0507 - val_accuracy: 0.7190
Ep

In [15]:
# Evaluation
score = model.evaluate(test_imgs, testY, verbose=0)
print("Accuracy: %.2f%%" % (score[1] * 100))

Accuracy: 53.07%


In [18]:
# Prediction
test_imgs = load_and_preprocess_images(test_data['id_code'], test_images_path)

In [19]:
# Make predictions using the model
predictions = model.predict(test_imgs)

61/61 [==============================] - 5s 85ms/step


In [20]:
# Extract the predicted class labels (assuming a classification task)
predicted_classes = predictions.argmax(axis=-1)

In [ ]:
# Create a submission file
submission = pd.DataFrame({'id_code': test_data['id_code'], 'diagnosis': predictions})
submission.to_csv("submission.csv", index=False)
print("Submission file created.")